In [1]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [15]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM

Read CSV create, and clean dataframe

In [16]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Team Statistics.csv")

mlb_teams

,YR,LG,Name,G,W,L,WSWin,Unnamed: 7,R,H,2B,3B,HR,BB,Unnamed: 14,SO,ERA,SHO,SV,FP
0,1969,NL,Atlanta Braves,162,93,69,N,NaN,691,1411,195,22,141,485,NaN,665,3.53,7,42,0.981
1,1969,AL,Baltimore Orioles,162,109,53,N,NaN,779,1465,234,29,175,634,NaN,806,2.83,20,36,0.984
2,1969,AL,Boston Red Sox,162,87,75,N,NaN,743,1381,234,37,197,658,NaN,923,3.92,7,41,0.975
3,1969,AL,California Angels,163,71,91,N,NaN,528,1221,151,29,88,516,NaN,929,3.54,9,39,0.978
4,1969,AL,Chicago White Sox,162,68,94,N,NaN,625,1346,210,27,112,552,NaN,844,4.21,10,25,0.981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2019,NL,St. Louis Cardinals,162,91,71,N,NaN,764,1336,246,24,210,561,NaN,1420,3.80,14,52,0.989
1404,2019,AL,Tampa Bay Rays,162,96,66,N,NaN,769,1427,291,29,217,542,NaN,1493,3.65,12,46,0.985
1405,2019,AL,Texas Rangers,162,78,84,N,NaN,810,1374,296,24,223,534,NaN,1578,5.06,9,33,0.982
1406,2019,AL,Toronto Blue Jays,162,67,95,N,NaN,726,1299,270,21,247,509,NaN,1514,4.79,7,33,0.984


In [17]:
#Get list of dataframe columns

mlb_teams.columns

Index(['YR', 'LG', 'Name', 'G', 'W', 'L', 'WSWin', 'Unnamed: 7', 'R', 'H',
       '2B', '3B', 'HR', 'BB', 'Unnamed: 14', 'SO', 'ERA', 'SHO', 'SV', 'FP'],
      dtype='object')

In [18]:
#Select columns for dataframe

mlb_df = mlb_teams[['YR', 'LG', 'Name', 'G', 'W', 'L', 'WSWin', 'R', 'H',
                   '2B', '3B', 'HR', 'BB', 'SO', 'ERA', 'SHO', 'SV', 'FP']]
mlb_df.head(20)

,YR,LG,Name,G,W,L,WSWin,R,H,2B,3B,HR,BB,SO,ERA,SHO,SV,FP
0,1969,NL,Atlanta Braves,162,93,69,N,691,1411,195,22,141,485,665,3.53,7,42,0.981
1,1969,AL,Baltimore Orioles,162,109,53,N,779,1465,234,29,175,634,806,2.83,20,36,0.984
2,1969,AL,Boston Red Sox,162,87,75,N,743,1381,234,37,197,658,923,3.92,7,41,0.975
3,1969,AL,California Angels,163,71,91,N,528,1221,151,29,88,516,929,3.54,9,39,0.978
4,1969,AL,Chicago White Sox,162,68,94,N,625,1346,210,27,112,552,844,4.21,10,25,0.981
5,1969,NL,Chicago Cubs,163,92,70,N,720,1400,215,40,142,559,928,3.34,22,27,0.979
6,1969,NL,Cincinnati Reds,163,89,73,N,798,1558,224,42,171,474,1042,4.11,11,44,0.973
7,1969,AL,Cleveland Indians,161,62,99,N,573,1272,173,24,119,535,906,3.94,7,22,0.976
8,1969,AL,Detroit Tigers,162,90,72,N,701,1316,188,29,182,578,922,3.31,20,28,0.979
9,1969,NL,Houston Astros,162,81,81,N,676,1284,208,40,104,699,972,3.60,11,34,0.975


In [19]:
#Set X(data) and y(team)variables

data = mlb_df.drop("Name", axis=1)
team = mlb_df["Name"]
print(data.shape, team.shape)

(1408, 17) (1408,)


In [20]:
#Use getdummies to convert strings

data_LG = data.copy()
data_bin = pd.get_dummies(data_LG, columns=["LG", "WSWin"])
data_bin = data_bin.drop(columns=['LG_AL','WSWin_N'])

data_bin.head()


,YR,G,W,L,R,H,2B,3B,HR,BB,SO,ERA,SHO,SV,FP,LG_NL,WSWin_Y
0,1969,162,93,69,691,1411,195,22,141,485,665,3.53,7,42,0.981,1,0
1,1969,162,109,53,779,1465,234,29,175,634,806,2.83,20,36,0.984,0,0
2,1969,162,87,75,743,1381,234,37,197,658,923,3.92,7,41,0.975,0,0
3,1969,163,71,91,528,1221,151,29,88,516,929,3.54,9,39,0.978,0,0
4,1969,162,68,94,625,1346,210,27,112,552,844,4.21,10,25,0.981,0,0


In [21]:
data_bin.count()

YR         1408
G          1408
W          1408
L          1408
R          1408
H          1408
2B         1408
3B         1408
HR         1408
BB         1408
SO         1408
ERA        1408
SHO        1408
SV         1408
FP         1408
LG_NL      1408
WSWin_Y    1408
dtype: int64

Preprocess data

In [22]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [23]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data_bin, team, random_state=1)
y_train

1114                  Cincinnati Reds
572                     New York Mets
154                Kansas City Royals
575                Pittsburgh Pirates
693                 Baltimore Orioles
                    ...              
715                  Seattle Mariners
905                   Cincinnati Reds
1096                    New York Mets
235                 Oakland Athletics
1061    Los Angeles Angels of Anaheim
Name: Name, Length: 1056, dtype: object

In [24]:
#Scale data using MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[0.82      , 0.96721311, 0.66666667, ..., 0.86956522, 1.        ,
        0.        ],
       [0.44      , 0.95081967, 0.48      , ..., 0.39130435, 1.        ,
        0.        ],
       [0.12      , 0.96721311, 0.66666667, ..., 0.34782609, 0.        ,
        0.        ],
       ...,
       [0.8       , 0.96721311, 0.38666667, ..., 0.69565217, 1.        ,
        0.        ],
       [0.18      , 0.96721311, 0.37333333, ..., 0.13043478, 0.        ,
        0.        ],
       [0.78      , 0.96721311, 0.78666667, ..., 0.73913043, 0.        ,
        0.        ]])

In [25]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_test

array([ 7, 34, 13,  6, 23, 10, 21, 21, 30, 18,  1, 20, 25, 28, 18,  4, 28,
        8, 19,  8, 30, 24,  6, 19, 22, 11, 24, 35, 24, 22, 24, 31, 21, 14,
       34, 26, 12,  3, 18, 24, 12, 14,  5, 33,  6, 30, 25, 34, 10, 26,  4,
       20, 30,  3,  5,  2, 26, 34, 12, 33, 33,  6,  2, 14, 28, 18, 22, 34,
       27, 19,  4, 11, 33,  8, 24,  5, 13,  0, 25,  2, 33, 22,  3, 21, 24,
       14,  6, 11, 26, 22, 13, 19,  6,  4, 19, 34, 18,  8, 21,  9, 16,  1,
       26, 20,  4, 11, 11, 34, 16,  2, 33,  9, 28,  4, 31, 21,  0, 16, 11,
       20,  4, 34, 30, 18, 11, 25, 25, 13, 33, 30, 32, 18, 33, 10, 18,  8,
       23, 23,  2, 18, 16, 19,  9,  1, 23,  7, 12, 34,  4,  6, 30, 24, 14,
       25,  8,  3, 18, 23,  9, 22,  5, 30, 34, 21, 16, 30, 13, 16, 17, 13,
       13,  6, 18, 22,  4,  2, 22,  9, 30, 16,  2,  9, 18,  5, 20, 10, 20,
        2, 16, 19, 24, 13, 18, 22,  1, 34, 13, 13, 12, 23, 30, 11, 32, 13,
       11,  3, 23,  2, 23, 20, 33, 12, 10, 26, 16,  7, 18,  9, 22, 18, 34,
       24, 18, 19,  4, 16

In [26]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.

Create Neural Network Model

In [46]:
# Create model and add layers

model.add(Dense(units=32, activation='relu', input_dim=17))
model.add(Dense(units=7, activation='relu'))
model.add(Dense(units=37, activation='softmax'))

In [47]:
#View model summary

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 231       
_________________________________________________________________
dense_2 (Dense)              (None, 37)                296       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 38        
_________________________________________________________________
dense_4 (Dense)              (None, 32)                64        
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 231       
_________________________________________________________________
dense_6 (Dense)              (None, 37)               

Train Model

In [48]:
#Compile model

model.compile(optimizer='adam',
              loss='mae',
              metrics=['accuracy'])

In [49]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=3
)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

Evaluate Model

In [112]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - loss: 3.7350 - accuracy: 0.0284
Deep Neural Network - Loss: 3.7349679470062256, Accuracy: 0.028409091755747795


Predict World Series Champs

In [ ]:
#Predict outcome